In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tensorflow.keras.layers import GRU, Flatten, Dense, Conv1D, Dropout, LeakyReLU
from tensorflow.keras import Sequential
from sklearn.metrics import mean_squared_error
from pathlib import Path
import os

%load_ext blackcellmagic

# %%black for formatting

2022-03-03 23:48:28.184794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-03 23:48:28.184811: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
load_path = Path(os.path.abspath('')).parents[0] / 'data' / 'scaled_data' 
load_path

PosixPath('/home/kuba1302/Gan/gan/data/scaled_data')

In [3]:
names = ['X_list', 'Y_preds_real_list', 'Y_whole_real_list']
def load_df_lists(names):
    data_dict = {}
    for name in names:
        with open(load_path / f'{name}.pickle', 'rb') as handle:
            data_dict[name] = pickle.load(handle)

data_dict = load_df_lists(names)

In [33]:
with open(load_path / 'X_list.pickle', 'rb') as test:
    X_list = pickle.load(test) 

In [5]:
data_dict

In [36]:
def generator(input_dim, feature_size, output_dim=1):
    model = Sequential()
    model.add(
        GRU(
            units=1024,
            return_sequences=True,
            input_shape=(input_dim, feature_size),
            recurrent_dropout=0.2,
        )
    )
    model.add(GRU(units=512, return_sequences=True, recurrent_dropout=0.2))
    model.add(GRU(units=256, return_sequences=False, recurrent_dropout=0.2))
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(units=output_dim))
    return model


def discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(
        Conv1D(
            32,
            input_shape=input_shape,
            kernel_size=3,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(
        Conv1D(
            64,
            kernel_size=5,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(
        Conv1D(
            128,
            kernel_size=5,
            strides=2,
            padding="same",
            activation=LeakyReLU(alpha=0.01),
        )
    )
    model.add(Flatten())
    model.add(Dense(220, use_bias=False))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(220, use_bias=False, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [7]:
class StockTimeGan:
    def __init__(self, batch_size, learning_rate, generator, discriminator):
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.generator = generator
        self.generator_optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.discriminator = discriminator
        self.discriminator_optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.checkpoint_directory = str(
            Path(os.path.abspath("").parents[0]) / "/checkpoints/"
        )
        self.checkpoint_prefix = str(Path(self.checkpoint_directory) / "ckpt")
        self.checkpoint = tf.train.Checkpoint(
            generator_optimizer=self.generator_optimizer,
            discriminator_optimizer=self.discriminator_optimizer,
            generator=self.generator,
            discriminator=self.discriminator,
        )

    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return self.cross_entropy(tf.ones_like(fake_output), fake_output)

    @tf.function
    def train_step(self, real_x, real_to_pred_y, real_whole_y): 
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_data = self.generator(real_x, training=True)
            

In [35]:
X_list.shape

(713, 30, 19)

In [9]:
test_x

,Open,High,Low,Volume,sma_5,ema_5,wma_5,bb_5_up,bb_5_down,sma_10,ema_10,wma_10,bb_10_up,bb_10_down,ema_20,ema_50,ema_100,macd,signal
0,0.075536,0.086009,0.082343,0.113326,0.078482,0.076267,0.078798,0.074062,0.123424,0.071822,0.074187,0.071794,0.081636,0.081858,0.068708,0.053655,0.044170,0.485153,0.499797
1,0.093843,0.072623,0.080214,0.133680,0.075105,0.069100,0.071309,0.083366,0.107309,0.068655,0.070112,0.068626,0.078760,0.078712,0.065803,0.051304,0.041952,0.474867,0.497355
2,0.087074,0.086634,0.082238,0.149944,0.075519,0.074075,0.074545,0.084464,0.106963,0.070012,0.072486,0.071342,0.081293,0.078651,0.067005,0.051955,0.042461,0.485516,0.497778
3,0.106150,0.110515,0.112766,0.150546,0.080936,0.084549,0.084697,0.101512,0.099651,0.072356,0.078529,0.077714,0.089321,0.074582,0.070708,0.054450,0.044640,0.507678,0.503061
4,0.134303,0.142126,0.142515,0.216692,0.091770,0.099439,0.100843,0.129053,0.091826,0.079014,0.087957,0.087972,0.107192,0.068377,0.076816,0.058716,0.048412,0.539999,0.514498
5,0.154302,0.146887,0.156759,0.123763,0.103878,0.113990,0.117943,0.153697,0.089445,0.090069,0.098245,0.099545,0.119759,0.076614,0.083814,0.063693,0.052826,0.573851,0.531200
6,0.156302,0.148136,0.159492,0.096799,0.122965,0.126487,0.133771,0.157556,0.121978,0.098036,0.108194,0.110636,0.136358,0.074376,0.090940,0.068849,0.057410,0.605142,0.551543
7,0.092920,0.075551,0.071959,0.675262,0.120958,0.108936,0.117731,0.162390,0.113122,0.097228,0.101584,0.106171,0.136629,0.072485,0.088169,0.067425,0.056267,0.581137,0.562462
8,0.088151,0.088663,0.092985,0.171906,0.117102,0.101272,0.106338,0.164998,0.103036,0.098020,0.098489,0.104053,0.136567,0.074118,0.087117,0.067056,0.056024,0.569299,0.568556
9,0.107535,0.091868,0.105446,0.132174,0.107129,0.093743,0.093841,0.160705,0.088397,0.098457,0.094597,0.100472,0.136362,0.075203,0.085359,0.066180,0.055339,0.555212,0.570288


In [37]:
generator = generator(X_list.shape[1], X_list.shape[2])

In [38]:
generator.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_5 (GRU)                 (None, 30, 1024)          3210240   
                                                                 
 gru_6 (GRU)                 (None, 30, 512)           2362368   
                                                                 
 gru_7 (GRU)                 (None, 256)               591360    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,205,185
Trainable params: 6,205,185
No

In [41]:
generator(X_list, training=True)

2022-03-04 00:20:43.886150: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 87613440 exceeds 10% of free system memory.
2022-03-04 00:20:43.901744: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 87613440 exceeds 10% of free system memory.
2022-03-04 00:20:43.926568: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 87613440 exceeds 10% of free system memory.
2022-03-04 00:20:45.314458: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 43806720 exceeds 10% of free system memory.
2022-03-04 00:20:45.321021: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 43806720 exceeds 10% of free system memory.


<tf.Tensor: shape=(713, 1), dtype=float32, numpy=
array([[-2.28117146e-02],
       [-1.97793320e-02],
       [-1.90764703e-02],
       [-2.09093578e-02],
       [-8.92277807e-04],
       [-1.57923810e-02],
       [-2.95880064e-03],
       [-1.60443708e-02],
       [-6.86994754e-03],
       [-1.26920640e-03],
       [ 1.81468278e-02],
       [-1.77361146e-02],
       [-6.91968948e-03],
       [ 3.66418809e-03],
       [-8.44436884e-03],
       [-1.56802796e-02],
       [-7.35656079e-03],
       [-9.45148617e-03],
       [-2.47131232e-02],
       [-5.94106317e-03],
       [-1.68713070e-02],
       [-1.10048382e-02],
       [-3.53754237e-02],
       [-1.63819063e-02],
       [-1.55841112e-02],
       [-3.05699818e-02],
       [-1.52754029e-02],
       [-1.12974411e-02],
       [-1.00354347e-02],
       [-7.99476728e-03],
       [-2.65906975e-02],
       [ 5.35177067e-03],
       [-1.12530766e-02],
       [ 4.57140990e-03],
       [-7.52642658e-03],
       [ 8.13340023e-03],
       [ 2.521